In [ ]:
!pip install dspy-ai
import dspy

In [ ]:
models=['Med-PaLM 2','mistral-7b-instruct','openchat-7b','gpt-3.5-turbo']
api_key = "API_KEY"

In [ ]:
def initialize_model(model_name):
    return dspy.OpenAI(model=model_name, api_key=api_key)

no_of_models=len(models)
for i in range(no_of_models):
    current_model = initialize_model(models[i])
    dspy.settings.configure(lm=current_model)

In [ ]:
!pip install datasets
from datasets import load_dataset
medqa_dataset = load_dataset("bigbio/med_qa")

In [ ]:
medqa_dataset = load_dataset("bigbio/med_qa")

#training on the first 20 questions
train_subset = medqa_dataset["train"][:20]

train_questions = train_subset["question"]
train_answers_id = train_subset["answer_idx"]
train_options= train_subset["options"]
train_answer = train_subset["answer"]

In [ ]:
print(train_questions[0])
print(train_options)
print(train_answer)
print(train_answers[0])

A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. Which of the following is the best treatment for this patient?
[[{'key': 'A', 'value': 'Ampicillin'}, {'key': 'B', 'value': 'Ceftriaxone'}, {'key': 'C', 'value': 'Ciprofloxacin'}, {'key': 'D', 'value': 'Doxycycline'}, {'key': 'E', 'value': 'Nitrofurantoin'}], [{'key': 'A', 'value': 'Placing the infant in a supine position on a firm mattress while sleeping'}, {'key': 'B', 'value': 'Routine postnatal electrocardiogram (ECG)'}, {'key': 'C', 'value': 'Keeping the infant covered 

In [ ]:
# We are expecting a single choice answer so signature accordingly.
class BasicQA(dspy.Signature):
    """Answer questions with one single letter"""

    question = dspy.InputField()
    answer = dspy.OutputField(desc="single letter")

In [ ]:
generate_answer = dspy.Predict(BasicQA)

def generate_responses(questions):
    responses = []
    for question in questions:
          generate_answer(question=question)
          generated_response = response.choices[0].message
          responses.append(generated_response)
    return responses

In [ ]:
generated_responses = generate_responses(train_questions)

In [ ]:
def calculate_medqa_accuracy(pred, true):
    correct_count = 0

    for pred, ref in zip(pred, true):
        if pred.strip().lower() == ref.strip().lower():
            correct_count += 1

    accuracy = correct_count / len(pred)
    return accuracy

accuracy = calculate_medqa_accuracy(generated_responses, train_answers_id)

print(f"\nAccuracy on the MedQA dataset: {accuracy * 100:.2f}%")

In [ ]:
# Now asking a question from the test dataset.
test_subset = medqa_dataset["test"]
test_question = test_subset["question"][0]
print(f"Predicted Answer: {generate_answer(question=test_question)}")